# News Article Classification

In [1]:
!pip install feedparser 
!pip install newspaper3k

  Obtaining dependency information for feedparser from https://files.pythonhosted.org/packages/7c/d4/8c31aad9cc18f451c49f7f9cfb5799dadffc88177f7917bc90a66459b1d7/feedparser-6.0.11-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.1 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=5695b53aa6b9b88a2ae47e9e7f0325610b70f219694d4e1b2921249da3c7739a
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 7.4 MB/s eta 0:00:00
  Obtaining dependency information for tldextract>=2.0.1 from https://files.pythonhosted.org/packages/d0/de/3f37b2568115c7ebeae39508dc1092f04f3dc286f22ef30171baca9c9cf2/tldextract-5.1.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/

In [2]:
import pandas as pd
import feedparser as fp
from  newspaper import Article
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
import pytz
import random

In [3]:
sources = {'terrorism': ['https://www.state.gov/rss-feed/counterterrorism/feed/',
                          'https://globalnews.ca/tag/terrorism/feed/',
                          'https://www.9news.com.au/terrorism/rss',],
           
 'political': ['https://globalnews.ca/politics/feed/',
              'https://www.aljazeera.com/xml/rss/all.xml',
              'https://globalnews.ca/politics/feed/' , 
              'https://www.propublica.org/feeds/propublica/main'],
           
 'protest': ['https://www.theguardian.com/world/protest/rss',
              'https://globalnews.ca/tag/protest/feed/'],
           
 'natural_disaster': ['https://www.theguardian.com/world/natural--disasters/rss',
                      'https://globalnews.ca/tag/natural-disasters/feed/',
                     'https://www.sciencedaily.com/rss/earth_climate/volcanoes.xml',
                     'https://www.sciencedaily.com/rss/earth_climate/floods.xml',
                     'https://www.sciencedaily.com/rss/earth_climate/earthquakes.xml',
                     'https://www.sciencedaily.com/rss/earth_climate/snow_and_avalanches.xml',
                     'https://www.sciencedaily.com/rss/earth_climate/landslides.xml',
                     'https://www.sciencedaily.com/rss/earth_climate/snow_and_avalanches.xml'
                     ],
           
 'positive': ['https://www.positive.news/feed/',
              'https://www.goodnewsnetwork.org/feed/'],
           
 'others': ['https://www.theguardian.com/science/rss',
              'https://globalnews.ca/science/feed/',
              'https://www.sciencenews.org/feed']}

In [4]:
def get_content(url):
    content = Article(url)
    content.download()
    content.parse()
    return content.text.replace("\n" , " ").lower()

In [5]:
get_content("https://globalnews.ca/news/10219172/rcmp-quebec-restauarant-stabbing-terrorism/")[:10]

'send this '

In [6]:
i = fp.parse('https://www.sciencedaily.com/rss/mind_brain/learning_disorders.xml')
i.entries[0].summary_detail.value

'No matter how well children are prepared for kindergarten, their transition to the classroom during the first few months plays a key role in their success, a new study suggests.'

In [7]:
i.entries[0].link

'https://www.sciencedaily.com/releases/2024/01/240122144343.htm'

In [8]:
def fetch_data(sources):
    main_ = { "title" : [] , "content" : [] , "label" : [] , "from_summary" : []}
    for class_ , li in sources.items():
        for i in li :
            try:
                source_ = fp.parse(i)
                if source_.bozo == False :
                    print(len(source_.entries))
                    for data in source_.entries : 
                        n = False
                        content = "hi"
                        try : 
                            content = get_content(data.link)
                        except Exception as e:
                            content = data.summary_detail.value
                            n = True
                        if content == '':
                            print(i.link)
                            content = data.summary_detial.value
                            n = True
                        main_["content"].append(content)
                        main_["from_summary"].append(n)
                        main_["label"].append(class_)
                        main_["title"].append(data.title.lower())
                elif source_.bozo == True :
                    print(i)
            except Exception as e:
                print(i , e)
    return main_

In [9]:
data = fetch_data(sources)

10
10
20
10
25
10
20
20
10
20
10
60
60
60
60
60
60
10
50
32
10
20


In [10]:
len(data["title"]), len(data["content"]), len(data["label"])

(647, 647, 647)

In [11]:
df = pd.DataFrame(data)
df.head()

,title,content,label,from_summary
0,taking financial measures against iraqi bank a...,the u.s. department of the treasury is today t...,terrorism,False
1,designating supporters of iran’s malign agenda,the united states is today designating iraqi a...,terrorism,False
2,targeting additional hamas financial networks,the united states is today imposing its fifth ...,terrorism,False
3,secretary antony j. blinken remarks to the press,"secretary blinken: good evening, everyone. so ...",terrorism,False
4,rewards for justice: reward offer for informat...,rewards for justice: reward offer for informat...,terrorism,False


In [12]:
value_counts = df.label.value_counts()

In [13]:
import plotly.express as px

fig = px.bar(x=value_counts.index, y=value_counts.values, labels={'x':'Category', 'y':'Count'})
fig.update_layout(title='Value Counts of Categories',
                  xaxis_title='Category',
                  yaxis_title='Count')

# Show the plot
fig.show()

In [14]:
import spacy
from nltk.stem.snowball import PorterStemmer , SnowballStemmer
from spacy.lang.en.stop_words import STOP_WORDS


stop_words_set = set(STOP_WORDS)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
stemmer_ = SnowballStemmer("english")

def remove_stopwords(text):
    content = []
    for word in text.lower().split():
        if word not in stop_words_set :
            content.append(word)
            
    return " ".join(content)


def stemmer(text):
    doc = nlp(text)
    stemmed_content = [stemmer_.stem(token.lemma_) for token in doc]
    return " ".join(stemmed_content)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning:

A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3



In [15]:
from collections import Counter
import string
def word_counts(word_list) :
    word_count = Counter()
    for i in word_list:
        word_count.update([j for j in i.split()])
        
    return word_count

def remove_puncs(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
    
    return text

In [16]:
df["soup"] = df["title"] + " " + df["content"].replace('continue reading' , " ")
df["soup"] = df["soup"].apply(remove_stopwords)
df["soup"]  = df["soup"].apply(stemmer)
df["soup"]  = df["soup"].apply(remove_puncs)

In [17]:
word_counts(df["soup"].to_list()).most_common()[:5]

[('say', 1780), ('’s', 1574), ('year', 862), ('peopl', 704), ('new', 669)]

In [18]:
classes = list(sources.keys())
for i in sources :
    temp = df.query(f" label == '{i}'")
    print(word_counts(temp["soup"].to_list()).most_common()[:6] , i)
    print()

[('say', 204), ('hama', 118), ('attack', 106), ('’s', 101), ('israel', 80), ('terrorist', 74)] terrorism

[('say', 638), ('’s', 604), ('state', 240), ('year', 237), ('not', 209), ('propublica', 200)] political

[('say', 168), ('protest', 150), ('’s', 127), ('peopl', 82), ('day', 70), ('polic', 53)] protest

[('say', 289), ('new', 287), ('research', 216), ('ice', 210), ('year', 192), ('flood', 192)] natural_disaster

[('’s', 312), ('year', 205), ('’', 151), ('say', 142), ('—', 133), ('time', 117)] positive

[('say', 339), ('’s', 297), ('’', 165), ('research', 148), ('it', 139), ('peopl', 138)] others



In [19]:
vocab = word_counts(df["soup"])
len(vocab)

13043

In [20]:
vocab_len = len(vocab)
vocab_len

13043

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.soup.to_list())

In [22]:
len(tokenizer.index_word)

13043

In [23]:
len_ = sorted([len(i.split()) for i in df.soup.to_list()] , reverse = True)

In [24]:
len_ = np.array(len_)

In [25]:
Counter(len_).most_common()[:5]

[(29, 18), (32, 16), (24, 15), (23, 15), (31, 13)]

In [26]:
def process_text(texts) : 
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen = 500 ,truncating='post', padding='post')
    return padded_sequences

In [27]:
shuffled_df = df.sample(frac=1).reset_index(drop=True)
shuffled_df.head()

,title,content,label,from_summary,soup
0,rescuers search for survivors after deadly ear...,at least 116 people have been killed and about...,natural_disaster,False,rescuer search survivor dead earthquak china –...
1,"israel’s war on gaza: list of key events, day 116",explainer israel’s war on gaza: list of key e...,political,False,israel ’s war gaza list key event day 116 ex...
2,plate tectonics not required for the emergence...,New finding contradicts previous assumptions a...,natural_disaster,True,plate tecton requir emerg life new find contra...
3,irs collected half a billion dollars in back t...,the internal revenue service says it collected...,positive,False,ir collect half billion dollar taxi delinqu mi...
4,volcanoes at fault if the earth slips,A new study has attributed the root cause of t...,natural_disaster,True,volcano fault earth slip new studi attribut ro...


In [28]:
sequences = process_text(shuffled_df.soup.to_list())
sequences

array([[2311, 1174, 1800, ...,    0,    0,    0],
       [  66,    2,  216, ...,    0,    0,    0],
       [ 999, 1443,  381, ...,    0,    0,    0],
       ...,
       [1052, 1063, 1086, ...,    0,    0,    0],
       [2449, 2610, 1392, ...,    0,    0,    0],
       [3287, 3225, 7206, ...,    0,    0,    0]], dtype=int32)

In [29]:
len(sequences[0])

500

In [30]:
classes

['terrorism', 'political', 'protest', 'natural_disaster', 'positive', 'others']

In [31]:
shuffled_df.head()

,title,content,label,from_summary,soup
0,rescuers search for survivors after deadly ear...,at least 116 people have been killed and about...,natural_disaster,False,rescuer search survivor dead earthquak china –...
1,"israel’s war on gaza: list of key events, day 116",explainer israel’s war on gaza: list of key e...,political,False,israel ’s war gaza list key event day 116 ex...
2,plate tectonics not required for the emergence...,New finding contradicts previous assumptions a...,natural_disaster,True,plate tecton requir emerg life new find contra...
3,irs collected half a billion dollars in back t...,the internal revenue service says it collected...,positive,False,ir collect half billion dollar taxi delinqu mi...
4,volcanoes at fault if the earth slips,A new study has attributed the root cause of t...,natural_disaster,True,volcano fault earth slip new studi attribut ro...


In [32]:
labels = np.array(shuffled_df.label.to_list()).reshape(-1 , 1)
labels[:3]

array([['natural_disaster'],
       ['political'],
       ['natural_disaster']], dtype='<U16')

In [33]:
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder

one_hot = OneHotEncoder(sparse = False)
labels = one_hot.fit_transform(labels )
labels

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [34]:
dict_ = { i:o for i,o in  enumerate(list(one_hot.categories_[0]))}
dict_

{0: 'natural_disaster',
 1: 'others',
 2: 'political',
 3: 'positive',
 4: 'protest',
 5: 'terrorism'}

In [35]:
from sklearn.model_selection import train_test_split

x_train, x_temp, y_train, y_temp = train_test_split(sequences , labels, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [36]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))

In [37]:
267/8

33.375

In [38]:
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(batch_size)
test_dataset = test_dataset.shuffle(buffer_size=10000).batch(batch_size)
val_dataset = val_dataset.shuffle(buffer_size=10000).batch(batch_size)

In [39]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D , Dropout , GlobalMaxPooling1D , LSTM

In [40]:
device = tf.test.gpu_device_name()
device

'/device:GPU:0'

In [41]:
with tf.device(device) :
#     model = Sequential([
#         Embedding(input_dim=vocab_len , output_dim = 64 ,   input_length=500),
#         GlobalMaxPooling1D(),
#         Dense(128, activation='relu'),
#         Dropout(0.5),  # Adjust dropout rate as needed
#         Dense(24, activation='relu'),
#         Dropout(0.5),  # Adjust dropout rate as needed
#         Dense(6, activation='softmax')  # Use 'softmax' for multi-class, 'sigmoid' for multi-label
#     ])
    model = Sequential([
            Embedding(input_dim=vocab_len, output_dim=64, input_length=500),
#             LSTM(64, return_sequences=True),  # Add LSTM layer
            GlobalMaxPooling1D(),
            Dense(128, activation='relu'),
            Dropout(0.5),  # Adjust dropout rate as needed
            Dense(24, activation='relu'),
            Dropout(0.5),  # Adjust dropout rate as needed
            Dense(6, activation='softmax')  # Use 'softmax' for multi-class, 'sigmoid' for multi-label
        ])
    
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])

In [42]:
from tensorflow.keras.callbacks import EarlyStopping , LearningRateScheduler , ModelCheckpoint , ReduceLROnPlateau , TensorBoard

early_stopping = EarlyStopping(monitor='val_accuracy', 
                                   patience=20, verbose=1, 
                                   mode='max', restore_best_weights=True)

model_checkpoint = ModelCheckpoint(filepath='model_article.h5', 
                                       monitor='val_accuracy', verbose=1, 
                                   save_best_only=True, mode='max')
    
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=2, min_lr=0.001)
    

lr_ind = []
def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        lr_ind.append(lr * tf.math.exp(-0.2))
        return lr * tf.math.exp(-0.2)
    
lr_schedule = LearningRateScheduler(scheduler)

In [43]:
with tf.device(device):
    model.fit(
        train_dataset ,
        epochs=300 ,
        callbacks=[
             lr_schedule #,early_stopping #, model_checkpoint  ,
        ],
    validation_data=val_dataset
    )

Epoch 1/300
15/15 [==============================] - 6s 227ms/step - loss: 1.5285 - accuracy: 0.5642 - val_loss: 1.3616 - val_accuracy: 0.5714 - lr: 0.0100
Epoch 2/300
15/15 [==============================] - 3s 209ms/step - loss: 1.3262 - accuracy: 0.6040 - val_loss: 1.2853 - val_accuracy: 0.5714 - lr: 0.0100
Epoch 3/300
15/15 [==============================] - 3s 195ms/step - loss: 1.0861 - accuracy: 0.6040 - val_loss: 0.9902 - val_accuracy: 0.5714 - lr: 0.0100
Epoch 4/300
15/15 [==============================] - 3s 168ms/step - loss: 0.7960 - accuracy: 0.6460 - val_loss: 0.8439 - val_accuracy: 0.6531 - lr: 0.0100
Epoch 5/300
15/15 [==============================] - 3s 181ms/step - loss: 0.7213 - accuracy: 0.6836 - val_loss: 0.8134 - val_accuracy: 0.6429 - lr: 0.0100
Epoch 6/300
15/15 [==============================] - 3s 194ms/step - loss: 0.6801 - accuracy: 0.7035 - val_loss: 0.8072 - val_accuracy: 0.6327 - lr: 0.0100
Epoch 7/300
15/15 [==============================] - 2s 138ms/st

In [44]:
model.save("model_article.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [45]:
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)